In [1]:
import os
from dotenv import load_dotenv
import dspy
from llama_index.core import (
    VectorStoreIndex, 
    StorageContext,
    load_index_from_storage,
    Document
)
load_dotenv()

dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

In [2]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [20]:
PERSIST_DIR = "lyrics_index"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)
retriever = index.as_retriever()


In [21]:
import wikipediaapi

def get_artist_wikipedia_info(artist: str) -> str:
    """
        Useful for getting artist information from wikipedia
        Use plain text as input to the tool, this should be the name of an artist or a band.
        The output of this tool is the summary of the wikipedia page corresponding to the artist.
    """
    wiki = wikipediaapi.Wikipedia("Test Wikipedia (eduardo.laruta@gmail.com)", "es")
    page = wiki.page(artist)
    if not page.exists():
        return f"Wikipedia page for {artist} does not exist"
    return f"{page.summary}\n{page.sections[0]}\n{page.sections[1]}"

def get_artist_relevant_song_lyrics(topic: str) -> list[str]:
    """
        Useful for getting song lyrics of Los Kjarkas. Use this tool for getting full text of
        song lyrics based on a topic.
        The output of this tool is a list of song lyrics.
    """
    nodes = retriever.retrieve(topic)
    return [node.text for node in nodes]

# display(Markdown(get_artist_wikipedia_info("tupay")))

In [22]:
class ContextQA(dspy.Signature):
    # context: str = dspy.InputField(desc="Context for the response, the context is composed of song lyrics")
    history: dspy.History = dspy.InputField()
    question: str = dspy.InputField()
    answer: str = dspy.OutputField(desc="Should be detailed, outputing excerpts from the original context")


class RAG(dspy.Module):
    def __init__(self, index):
        self.respond = dspy.ReAct(
            signature=ContextQA, 
            tools=[get_artist_wikipedia_info, get_artist_relevant_song_lyrics],
            max_iters=4
        )
        self.refine_question = dspy.Predict("question -> main_topic")
        self.chat_history = dspy.History(messages=[])
        # self.retriever = index.as_retriever()

    def forward(self, question):
        # refined_question = self.refine_question(question=question).main_topic
        # print(f"refined question: {refined_question}")
        # context = "\n".join([node.text for node in self.retriever.retrieve(refined_question)])
        # print(f"context: {context}")
        outputs = self.respond(history=self.chat_history, question=question)
        self.chat_history.messages.append({"question": question, **outputs})
        return outputs.answer
    


In [23]:
rag = RAG(index)

In [24]:
rag(question="cuales son las canciones de protesta de los kjarkas?")

'Las canciones de protesta de Los Kjarkas incluyen:\n\n1. **"El líder de los humildes"**: Esta canción habla sobre un hombre que busca la libertad y lucha por los derechos de los pobres, enfrentándose a un sistema opresor. La letra destaca la lucha de los oprimidos y la búsqueda de una vida digna: "librara su guerra de hambre, tras muerte y rebelión, luchando contra un sistema que quisieron derrumbar".\n\n2. **"Contra el viento"**: En esta canción, se expresa la lucha por la libertad y la búsqueda de razones para seguir adelante a pesar de las adversidades. La letra menciona: "por que pienso diferente, yo no sé si es mi pecado, el haber nacido libre y libre querer seguir", reflejando la importancia de la libertad de pensamiento y la resistencia ante la opresión.\n\nAmbas canciones son ejemplos claros del compromiso de Los Kjarkas con la justicia social y la protesta contra las injusticias.'

In [25]:
rag(question="quienes son los kjarkas?")

'Los Kjarkas son un conjunto de música folklórica originario de Capinota, Cochabamba, Bolivia, formado por los hermanos Wilson, Gonzalo, Castel Hermosa y Edgar Villaroel. Su nombre, que proviene del Quechua, significa "Los Valientes" o "Los con Coraje". Desde su fundación en la década de 1970, han alcanzado una gran popularidad tanto en Bolivia como en el extranjero.\n\nA lo largo de su historia, Los Kjarkas han evolucionado musicalmente, adaptando ritmos autóctonos a estructuras más amplias y han sido pioneros en la difusión de la música andina. Han lanzado numerosos álbumes y canciones icónicas, como "Llorando se fue" y "Wayayay", que han dejado una huella significativa en la música latinoamericana. \n\nEl grupo ha experimentado varios cambios en su formación, pero siempre ha mantenido su esencia folklórica. Su legado continúa vivo, y siguen siendo un símbolo de la música folklórica boliviana, innovando y adaptándose a los cambios en la industria musical.'

In [26]:
rag(question="por qué son tan famosos?")

'Los Kjarkas son famosos por su innovadora fusión de ritmos autóctonos bolivianos con estructuras musicales más amplias, lo que les ha permitido alcanzar una gran popularidad tanto en Bolivia como a nivel internacional. Desde sus inicios, han trabajado para perfeccionar su estilo, creando un sonido distintivo que combina instrumentos tradicionales con arreglos corales.\n\nSu participación en festivales internacionales, como el Festival de Yamaha de la Canción, les ha proporcionado una plataforma para mostrar su talento. La canción "Florcita azul", compuesta por Ulises Hermosa, alcanzó el décimo lugar en este festival, lo que les abrió las puertas al mercado japonés y aumentó su visibilidad global.\n\nAdemás, el incidente con la canción "Llorando se fue", que fue plagiada por el grupo Kaoma en su éxito "Chorando se foi", les otorgó reconocimiento internacional y resultó en una indemnización que elevó su perfil en la industria musical.\n\nA lo largo de su carrera, han lanzado numerosos á

In [27]:
rag.inspect_history()





[2025-10-23T08:08:22.310674]

System message:

Your input fields are:
1. `history` (History): 
2. `question` (str): 
3. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Should be detailed, outputing excerpts from the original context
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## history ## ]]
{history}

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `history`, `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
cuales son las canciones de protesta de los kjarkas?

[[ ## trajectory ## ]]
{"thought_0": "Para responder a la pregunta sobre las canciones de protesta de Los Kjarkas, sería útil obtener las letras de las canciones que abordan temas de protesta. Por lo tanto, voy a buscar 

In [28]:
rag(question="recomiendame algun grupo similar a los kjarkas y dame informacion resumida sobre el grupo")

'Te recomiendo el grupo **Los Huayra**, un conjunto argentino de balada folclórica que se formó en Salta en 1995 y se disolvió en 2019. La banda estaba integrada por Juan José Vasconcellos, Juan Fuentes, Sebastián Giménez, Luis Benavidez, Hernando Mónico y Álvaro Plaza. \n\nLos Huayra lanzaron su álbum debut en 2005, que llevó el nombre de la banda, y rápidamente ganaron reconocimiento, obteniendo el premio Revelación en el Festival Nacional de Doma y Folklore de Jesús María. Su segundo álbum, "La voz del viento" (2007), incluye su canción más popular, "Una noche sin ti". A lo largo de su carrera, también lanzaron otros discos como "Código de barro" (2009), "Pueblo" (2011), y "Gira" (2016), que fue nominado a los Premios Grammy Latinos.\n\nEl grupo se destacó por su capacidad de fusionar ritmos tradicionales con un enfoque moderno, lo que les permitió conectar con una amplia audiencia. Tras su separación, algunos de sus miembros continuaron en la música, formando nuevas bandas o lanzan

In [29]:
rag.inspect_history()





[2025-10-23T08:11:47.552842]

System message:

Your input fields are:
1. `history` (History): 
2. `question` (str): 
3. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Should be detailed, outputing excerpts from the original context
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## history ## ]]
{history}

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `history`, `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
cuales son las canciones de protesta de los kjarkas?

[[ ## trajectory ## ]]
{"thought_0": "Para responder a la pregunta sobre las canciones de protesta de Los Kjarkas, sería útil obtener las letras de las canciones que abordan temas de protesta. Por lo tanto, voy a buscar 